___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [401]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [402]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[@pedroazambuja14]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [403]:
#Dados de autenticação do twitter: 

#Coloque aqui o identificador da conta no twitter: @pedroazambuja14

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [404]:
#Produto escolhido:
produto = 'macbook'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [405]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [406]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [407]:
dados = pd.ExcelFile("macbook.xlsx",sep=",")
dados.sheet_names
Teste = dados.parse("Teste")
Treinamento = dados.parse("Treinamento")
Treinamento.Classificacao.describe()

#Renomeando classificacao para relevante e irrelevante
Treinamento.Classificacao = Treinamento.Classificacao.astype("category")
Treinamento.Classificacao.cat.categories = (["irrelevante", "relevante"])

Teste.Classificacao = Teste.Classificacao.astype("category")
Teste.Classificacao.cat.categories = (["irrelevante", "relevante"])

#Transformando em objeto para evitar erros de código
Treinamento.Classificacao = Treinamento.Classificacao.astype("object")
Teste.Classificacao = Teste.Classificacao.astype("object")

In [422]:
#Limpeza em Teste

Teste.Teste = Teste.Teste.replace(":","",regex=True)
Teste.Teste = Teste.Teste.replace(","," ",regex=True)
Teste.Teste = Teste.Teste.replace("()","",regex=True)
Teste.Teste = Teste.Teste.replace("-","",regex=True)
Teste.Teste = Teste.Teste.replace("@","",regex=True)
Teste.Teste = Teste.Teste.replace("ela","",regex=True)
Teste.Teste = Teste.Teste.replace("!","",regex=True)
Teste.Teste = Teste.Teste.replace("rt","",regex=True)
Teste.Teste = Teste.Teste.replace("\n","",regex=True)
Teste.Teste = Teste.Teste.replace("#","",regex=True)
Teste.Teste = Teste.Teste.replace("mas","",regex=True)
Teste.Teste = Teste.Teste.replace("-","",regex=True)


In [423]:
#Limpeza em Treinamento

Treinamento.Treinamento = Treinamento.Treinamento.replace(":","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace(","," ",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("()","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("-","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("@","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("ela","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("!","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("rt","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("\n","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("#","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("mas","",regex=True)
Treinamento.Treinamento = Treinamento.Treinamento.replace("-","",regex=True)



Treinamento

,Treinamento,Classificacao
0,larissadare brunaprescovia real vendeu um ma...,irrelevante
1,não ligo tanto para as paradas da apple esse ...,relevante
2,gostei de um vídeo youtube https//t.co/aerirkk...,irrelevante
3,e fuckmylife porque comprei o macbook pro e es...,relevante
4,gostei de um vídeo youtube https//t.co/xswis8y...,irrelevante
5,bringmethegu eu achava q era caro os celulares...,relevante
6,que vergonha e ainda quebrei a capa do macbook...,irrelevante
7,tec_mundo [melhores da semana] hackers conseg...,irrelevante
8,o cruzeiro_news vendeu o macbook. o universo v...,irrelevante
9,quero um macbook,relevante


## Encontrando P(relevante) e P(irrelevante)

In [410]:
irrelevante = Treinamento.Classificacao.value_counts()[0]
relevante = Treinamento.Classificacao.value_counts()[1]

P_relevante = relevante/(relevante+irrelevante)
P_irrelevante = irrelevante/(relevante+irrelevante)

Teste.Classificacao.value_counts()

irrelevante    140
relevante       60
Name: Classificacao, dtype: int64

## Encontrando a frequência das palavras

In [411]:
######Agrupamento de palavras sinonimas######
querer = ["queria","quer","quero","querem","queriam"]
######Relevantes######
Relevante = Treinamento[Treinamento.Classificacao == "relevante"]
R = []
dic_R = {}

for i in Relevante.Treinamento:
    R.append(i.split())
for i in R:
    for e in i:
        if e in dic_R.keys():
            dic_R[e] += 1
        else:
            dic_R[e] = 1
            

    
######Irrelevantes######        
Irrelevante = Treinamento[Treinamento.Classificacao == "irrelevante"]
I = []
dic_I = {}

for i in Irrelevante.Treinamento:
    I.append(i.split())
for i in I:
    for e in i:
        if e in dic_I.keys():
            dic_I[e] += 1
        else:
            dic_I[e] = 1
            
#####Dicionário de Probabilidades P(palavra/relevante) ou P(palavra/irrelevante)######  

#relevantes
dic_freq_R = dic_R          
for i in dic_R:
    dic_freq_R[i] = (dic_R[i]+1)/(2*len(dic_R) + len(dic_I))
    
#irrelevantes
dic_freq_I = dic_I          
for i in dic_I:
    dic_freq_I[i] = (dic_I[i]+1)/(len(dic_R) + 2*len(dic_I))
    
    


## Função Classificadora

In [412]:
##Função que fornece uma Lista classificada
def Probabilidade(comentarios,dic_freq_R,dic_freq_I,P_relevante,P_irrelevante):
    Lista_final = []
    for i in comentarios:
        P_r = P_relevante
        P_i = P_irrelevante
        for e in i:
            #Probabilidade de ser relevante
            if e in dic_freq_R.keys():         
                P_r *= dic_freq_R[e]
            else:
                P_r *= 1/(len(dic_freq_I) + len(dic_freq_R))
            #Probabilidade de ser irrelevante
            if e in dic_freq_I.keys():     
                P_i *= dic_freq_I[e]
            else:
                P_i *= 1/(len(dic_freq_I) + len(dic_freq_R))
        if P_r >= P_i:
            Lista_final.append([i,"relevante"])
        if P_i>P_r:
            Lista_final.append([i,"irrelevante"])
    return Lista_final


##Função Classificadora
def Classificador(df,dic_freq_R,dic_freq_I,P_relevante,P_irrelevante):
    Todos = []
    for i in df.Teste:
        Todos.append(i.split())
    x = Probabilidade(Todos,dic_freq_R,dic_freq_I,P_relevante,P_irrelevante)
    #Transformando lista de palavras em uma sentença
    s = " "
    for e in x:
        e[0] = s.join(e[0])
    y=[]
    z=[]
    for i in x:
        y.append(i[0])
        z.append(i[1])

    #Criando DataFrame classificado pelo Programa
    classificado = pd.DataFrame()
    classificado["Comentário"] = pd.Series(y)
    classificado["Auto_Classificacao"] = pd.Series(z)
    return classificado

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [420]:
df = Classificador(Teste,dic_freq_R,dic_freq_I,P_relevante,P_irrelevante)


## Colocando classificacoes lado a lado para avaliar performance

In [421]:
df["Classificacao"] = Teste.Classificacao
df

,Comentário,Auto_Classificacao,Classificacao
0,eu tô chorando por que meu pai pegou meu macbo...,irrelevante,relevante
1,diogot dougdiskin grigio e esse adaptador pro ...,irrelevante,irrelevante
2,emersontsilva bom diafiz atualização do ios si...,irrelevante,irrelevante
3,a menina falando da um salve na quebrada atras...,irrelevante,relevante
4,larissadare brunaprescovia poanto foi falha do...,irrelevante,irrelevante
5,stephanny você quer viajar? mas ela disse q qu...,irrelevante,relevante
6,"não uso, tenho (celinha com s5 e iphone da fil...",irrelevante,relevante
7,"guizolio drogado de fruity loops, pra que no f...",irrelevante,relevante
8,depois de quase 4 anos eu descobri aonde fica ...,irrelevante,irrelevante
9,serei assim se eu ganhar um macbook dourado ht...,irrelevante,relevante


In [418]:
L_performance_relevante = []
L_performance_irrelevante = []


for i in range(len(df)):
    if df.Classificacao[i] == 'relevante':
        if df.Auto_Classificacao[i] == 'relevante':
            L_performance_relevante.append('positivos verdadeiros')
        elif df.Auto_Classificacao[i] == 'irrelevante':
            L_performance_relevante.append('negativos falsos')
    elif df.Classificacao[i] == 'irrelevante':
        if df.Auto_Classificacao[i] == 'irrelevante':
            L_performance_irrelevante.append('negativos verdadeiros')
        elif df.Auto_Classificacao[i] == 'relevante':
            L_performance_irrelevante.append('positivos falsos')
            
            
porcentagem_relevante = pd.Series(L_performance_relevante)
a = (porcentagem_relevante.value_counts())/60 #total de comentarios relevante

porcentagem_irrelevante = pd.Series(L_performance_irrelevante)
b = (porcentagem_irrelevante.value_counts())/140

print(a)
print("\n")
print(b)

negativos falsos         0.8
positivos verdadeiros    0.2
dtype: float64


negativos verdadeiros    0.957143
positivos falsos         0.042857
dtype: float64


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


### conclusao

Partindo da performance obtida, é possível notar que o programa tende a classificar a maioria das mensagens como irrelevantes(negativos verdadeiros = 95%, enquanto positivos verdadeiros = 20%). 

Um dos principais motivos desse resultado se deve ao fato de que a principal palavra chave do tema analisado, "macbook", se encontra em absolutamente todos os comentários, o que acaba relevando a sua importância no momento da classificação(essa palavra deixa de ter um peso significativo). Assim, a sua precisão acaba sendo prejudicada pelo fato de que a maioria dos comentários sobre esse produto são irrelevantes(+- 70%), o que compromete o processo de machine learning.

Outro Motivo para isso é que não existe um meio de tratamento de mensagens com dupla negação e sarcasmo, há apenas a filtragem de termos que não agregam na classificação(como: @, "e", "#" e etc.).

Apesar disso, o programa mostra grande potencial para a classificação adequada das mensagens. Com futuras iterações serão implementadas técnicas avançadas de filtragem, que agruparão palavras sinônimas(como: "quer" e "queria") em um uníco grupo, o que significativamente aumentará  a frequencia dessas palavras, o que com certeza resultará em melhores resultados dessa análise.

Por Outro lado, foi possível observar uma tendência de comentários do tipo: quero um macbook, preciso de um macbook etc. Isso significa que esse programa classificador foi capaz de encontrar pessoas que querem comprar o macbook da apple. Com essa informação, é possível expor esses usúario às propragandas do macbook, o que provavelmente aumentará as vendas do laptop.